# Benchmark NICE as calibration method

In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
import os
import sys
import time
import importlib
import functools
import concurrent
import collections
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.isotonic import IsotonicRegression
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Input
import tensorflow.keras.backend as K

pd.set_option('colheader_justify', 'center')

%aimport utils
%aimport utils.ops
%aimport utils.metrics
%aimport utils.visualization
%aimport utils.data
%aimport flows.nice
%aimport flows.normalizing_flows
%aimport calibrators
from utils.ops import onehot_encode, optim_temperature, detection_log_likelihood_ratios
from utils.metrics import neg_log_likelihood, accuracy, expected_calibration_error
from utils.visualization import plot_pdf_simplex, plot_prob_simplex, reliability_plot, ECE_plot
from utils.data import get_cifar10
from flows.nice import NiceFlow
from calibrators import DummyCalibrator, PAVCalibrator, TempScalingCalibrator, MLRCalibrator
from calibrators import NiceCalibrator, PlanarFlowCalibrator, RadialFlowCalibrator, RealNvpCalibrator

C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\f

In [4]:
def highlight_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

In [5]:
def highlight_min(s):
    is_min = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_min]

### Helper methods:

In [6]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [7]:
def score(calibrator, logits, target):
    probs = calibrator.predict(logits)
    nll = neg_log_likelihood(probs, target)
    ece = expected_calibration_error(probs, target, bins=15)
    acc = accuracy(probs, target)
    
    metrics = {
        'NLL': nll,
        'ECE': ece,
        'Accuracy': acc,
    }
    
    return metrics

In [8]:
def train_calibrator(Calibrator, logits, target):
    cal = Calibrator(logits, target)
    return cal

In [9]:
if len(get_available_gpus())>1:
    def train_and_evaluate_calibrators(gpu, logits, target, test_logits, test_target, calibrators, **kwargs):
        
        ## Train NICE on test set to obtain NLLmin
        with tf.Session(config=tf.ConfigProto(allow_soft_placement=True), graph=tf.Graph()) as sess:
            K.set_session(sess)
            with tf.device(gpu):
                nice_ref_cal = NiceCalibrator(test_logits, test_target, **kwargs['nice_ref_args'])
                ref_results = score(nice_ref_cal, test_logits, test_target)

        results = collections.OrderedDict()
        for cal, Calibrator in calibrators.items():
            with tf.Session(config=tf.ConfigProto(allow_soft_placement=True), graph=tf.Graph()) as sess:
                K.set_session(sess)
                with tf.device(gpu):
                    t0 = time.time()
                    model = train_calibrator(Calibrator, logits, target)
                    t1 = time.time() - t0
                    print("Calibrator {} fitted in {:.2f}s".format(cal, t1))
                    results[cal] = {'Training time': t1,
                                    'Validation': score(model, logits, target),
                                    'Test': score(model, test_logits, test_target)}

        return results, ref_results
    
else:
    
    def train_and_evaluate_calibrators(logits, target, test_logits, test_target, calibrators, **kwargs):
        ## Train NICE on test set to obtain NLLmin
        nice_ref_cal = NiceCalibrator(test_logits, test_target, **kwargs['nice_ref_args'])
        ref_results = score(nice_ref_cal, test_logits, test_target)

        results = collections.OrderedDict()
        for cal, Calibrator in calibrators.items():
            t0 = time.time()
            model = train_calibrator(Calibrator, logits, target)
            t1 = time.time() - t0
            print("Calibrator {} fitted in {:.2f}s".format(cal, t1))
            results[cal] = {'Training time': t1,
                            'Validation': score(model, logits, target),
                            'Test': score(model, test_logits, test_target)}

        return results, ref_results

In [10]:
def calibrate_models_gpu(args):
    gpu, models = args
    for model in models:
        print("Calibrating model: {}".format(model))
        data_path = os.path.join('../data', model+'_cifar100')
        prefix = os.path.join(data_path, 'cifar100_'+model)

        logits = np.load(prefix + '_logit_prediction_valid.npy')
        test_logits = np.load(prefix + '_logit_prediction_test.npy')

        target = np.load(prefix + '_true_valid.npy')
        test_target = np.load(prefix + '_true_test.npy')

        results[model], ref_results[model] = train_and_evaluate_calibrators(gpu,
                                                                            logits, 
                                                                            target, 
                                                                            test_logits, 
                                                                            test_target, 
                                                                            calibrators,
                                                                            **kwargs)
    return results, ref_results

## CIFAR-100

In [11]:
gpus = get_available_gpus()

models = [
    'wide-resnet-28x10',
    'densenet-121',
    'densenet-169',
    'resnet-101',
    'vgg-19',
    'preactresnet-18',
    'preactresnet-164',
    'resnext-29_8x16',
    'wide-resnet-40x10',
]
models = [
    'wide-resnet-28x10',
    'densenet-121',
    'densenet-169',
    'resnet-101',
]

In [12]:
results = collections.OrderedDict()
ref_results = collections.OrderedDict()

calibrators = {
    'Uncalibrated': DummyCalibrator,
    'Temp-Scaling': TempScalingCalibrator,
    'MLR': MLRCalibrator,
}

nice_args = [
    {
        'layers': 2,
        'hidden_size': [1],
        'epochs': 500,
    }, {
        'layers': 2,
        'hidden_size': [5],
        'epochs': 500,
    }, {
        'layers': 4,
        'hidden_size': [100, 100],
        'epochs': 1000,
    },
]

nvp_args = [
    {
        'layers': 2,
        'hidden_size': [1],
        'epochs': 500,
    }, {
        'layers': 2,
        'hidden_size': [5],
        'epochs': 500,
    }, {
        'layers': 4,
        'hidden_size': [100, 100],
        'epochs': 1000,
    },
]

planar_args = [
    {
        'layers': 5,
        'epochs': 2000,
    },{
        'layers': 10,
        'epochs': 2000,
    },{
        'layers': 20,
        'epochs': 2000,
    }
]

radial_args = [
    {
        'layers': 5,
        'epochs': 2000,
    },{
        'layers': 10,
        'epochs': 2000,
    },{
        'layers': 20,
        'epochs': 2000,
    }
]

    
"""
for nice in nice_args:
    name = 'NICE_l{}_hs{}'.format(nice['layers'], nice['hidden_size'])
    calibrators[name] = lambda logits, target: NiceCalibrator(logits, target, **nice)
    
for nvp in nvp_args:
    name = 'RealNVP_l{}_hs{}'.format(nvp['layers'], nvp['hidden_size'])
    calibrators[name] = lambda logits, target: RealNvpCalibrator(logits, target, **nvp)

for planar in planar_args:
    name = 'Planar_l{}'.format(planar['layers'])
    calibrators[name] = lambda logits, target: PlanarFlowCalibrator(logits, target, **planar)
"""
for radial in radial_args:
    name = 'Radial_l{}'.format(radial['layers'])
    calibrators[name] = lambda logits, target: RadialFlowCalibrator(logits, target, **radial)
    

nice_ref_args = {
    'layers': 4,
    'hidden_size': [100, 100],
    'epochs': 1000,
}

kwargs = {
    'nice_ref_args': nice_ref_args,
}

if len(gpus)<2:
    for model in models:
        data_path = os.path.join('../data', model+'_cifar100')
        prefix = os.path.join(data_path, 'cifar100_'+model)

        logits = np.load(prefix + '_logit_prediction_valid.npy')
        test_logits = np.load(prefix + '_logit_prediction_test.npy')

        target = np.load(prefix + '_true_valid.npy')
        test_target = np.load(prefix + '_true_test.npy')

        results[model], ref_results[model] = train_and_evaluate_calibrators(logits, 
                                                                            target, 
                                                                            test_logits, 
                                                                            test_target, 
                                                                            calibrators,
                                                                            **kwargs)
else:
    n = len(models)//len(gpus)
    zipped_models = [models[i*n:(i+1)*n] for i in range((len(models) + n - 1) // n)]
    with concurrent.futures.ThreadPoolExecutor(len(gpus)) as executor:
        results, results_ref = functools.reduce(lambda x, y: [{**x[0], **y[0]}, {**x[1], **y[1]}], 
                                                [(r, r_ref) for r, r_ref in executor.map(calibrate_models_gpu, zip(gpus, zipped_models))],
                                                [{}, {}])
        print('results: ', results)


Calibrating model: wide-resnet-28x10
Calibrating model: densenet-169
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Calibrator Uncalibrated fitted in 0.00s
Calibrator Temp-Scaling fitted in 1.01s
Calibrator MLR fitted in 5.24s
Calibrator Uncalibrated fitted in 0.00s
Calibrator Temp-Scaling fitted in 4.03s
Calibrator MLR fitted in 11.05s
Calibrator Radial_l5 fitted in 1421.64s
Calibrator Radial_l5 fitted in 1445.25s
Calibrator Radial_l10 fitted in 1446.86s


Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E354F64048>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.
Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E35EEA06A0>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi

Calibrator Radial_l10 fitted in 1455.74s


Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E35EE83128>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.
Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E365C2B320>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi

Calibrator Radial_l20 fitted in 1482.68s
Calibrating model: densenet-121
Calibrator Radial_l20 fitted in 1494.32s
Calibrating model: resnet-101
Calibrator Uncalibrated fitted in 0.00s
Calibrator Temp-Scaling fitted in 4.04s
Calibrator MLR fitted in 5.88s


Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E0B66B9438>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.
Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E0B6805128>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi

Calibrator Uncalibrated fitted in 0.00s
Calibrator Temp-Scaling fitted in 3.94s
Calibrator MLR fitted in 6.82s
Calibrator Radial_l5 fitted in 1407.75s
Calibrator Radial_l5 fitted in 1427.54s
Calibrator Radial_l10 fitted in 1419.02s
Calibrator Radial_l10 fitted in 1442.50s


Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E379D0D048>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.
Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001E37DD4B320>>
Traceback (most recent call last):
  File "C:\Users\sergi\Anaconda3\envs\calib\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sergi

Calibrator Radial_l20 fitted in 1431.20s
Calibrator Radial_l20 fitted in 1445.92s
results:  {'wide-resnet-28x10': OrderedDict([('Uncalibrated', {'Training time': 0.002991914749145508, 'Validation': {'NLL': 0.8195012069079306, 'ECE': 0.05312395484149458, 'Accuracy': 0.7994}, 'Test': {'NLL': 0.817314659817216, 'ECE': 0.048530271295458054, 'Accuracy': 0.8039}}), ('Temp-Scaling', {'Training time': 1.0063085556030273, 'Validation': {'NLL': 0.8156984518259202, 'ECE': 0.04849923598095774, 'Accuracy': 0.7994}, 'Test': {'NLL': 0.8134477968198074, 'ECE': 0.042847461207210995, 'Accuracy': 0.8039}}), ('MLR', {'Training time': 5.238993167877197, 'Validation': {'NLL': 0.7780407887568063, 'ECE': 0.04374047675362761, 'Accuracy': 0.8064}, 'Test': {'NLL': 0.8010431549744933, 'ECE': 0.04228785558311745, 'Accuracy': 0.8041}}), ('Radial_l5', {'Training time': 1421.6434490680695, 'Validation': {'NLL': 0.7105920608332322, 'ECE': 0.027069147253036527, 'Accuracy': 0.8136}, 'Test': {'NLL': 0.7717742568379763, '

In [13]:
ece_val_results = {}
ece_test_results = {}

acc_val_results = {}
acc_test_results = {}

nll_val_results = {}
nll_test_results = {}

for model, model_results in results.items():
    ece_val_results[model] = {}
    ece_test_results[model] = {}
    acc_val_results[model] = {}
    acc_test_results[model] = {}
    nll_val_results[model] = {}
    nll_test_results[model] = {}
    for cal, cal_results in model_results.items():
        ece_test_results[model][cal] = cal_results['Test']['ECE']
        ece_val_results[model][cal] = cal_results['Validation']['ECE']
        
        acc_test_results[model][cal] = cal_results['Test']['Accuracy']
        acc_val_results[model][cal] = cal_results['Validation']['Accuracy']
        
        nll_test_results[model][cal] = cal_results['Test']['NLL']
        nll_val_results[model][cal] = cal_results['Validation']['NLL']

**Results on the test set:**

In [14]:
df = pd.concat([pd.DataFrame.from_dict(acc_test_results, orient='columns'),
                pd.DataFrame.from_dict(ece_test_results, orient='columns'),
                pd.DataFrame.from_dict(nll_test_results, orient='columns')],
               axis=1,keys=['ACC','ECE', 'NLL']).swaplevel(0,1,axis=1).sort_index(axis=1)

df = df[models].loc[list(calibrators.keys())]
df.style.set_properties(**{'text-align': 'center'})\
    .set_caption('CIFAR100 Test set')\
    .apply(highlight_max, subset=[(model, 'ACC') for model in df.columns.levels[0]])\
    .apply(highlight_min, subset=[(model, 'ECE') for model in df.columns.levels[0]])\
    .apply(highlight_min, subset=[(model, 'NLL') for model in df.columns.levels[0]])\
    .set_table_styles([dict(selector="th", props=[("text-align", "center")]),
                       dict(selector="caption", props=[("text-align", "center"),
                                                      ("font-size", "200%"),
                                                      ("color", "black")])])

In [15]:
nll_df = pd.DataFrame.from_dict(nll_test_results, orient='columns')
min_nll_df = pd.DataFrame.from_dict(ref_results, orient='columns').loc['NLL']
nll_cal_df = nll_df.subtract(min_nll_df, axis='columns')

df = pd.concat([nll_df, nll_cal_df], axis=1,keys=['NLL','NLL_cal']).swaplevel(0,1,axis=1).sort_index(axis=1)
df = df[models].loc[list(calibrators.keys())]

df.rename(columns={model: model + ' NLL_min={:.4f}'.format(min_nll_df[model]) 
                   for model in df.columns.levels[0]}, level=0, inplace=True)

df.style.set_properties(**{'text-align': 'center'})\
    .format("{:.4f}")\
    .set_caption('CIFAR100 NLL decomposition')\
    .apply(highlight_min)\
    .set_table_styles([dict(selector="th", props=[("text-align", "center")]),
                       dict(selector="caption", props=[("text-align", "center"),
                                                      ("font-size", "200%"),
                                                      ("color", "black")])])

**Results on the validation set:**

In [16]:
df = pd.concat([pd.DataFrame.from_dict(acc_val_results, orient='columns'),
                pd.DataFrame.from_dict(ece_val_results, orient='columns'),
                pd.DataFrame.from_dict(nll_val_results, orient='columns')],
               axis=1,keys=['ACC','ECE', 'NLL']).swaplevel(0,1,axis=1).sort_index(axis=1)

df = df[models].loc[list(calibrators.keys())]
df.style.set_properties(**{'text-align': 'center'})\
    .set_caption('CIFAR100 Validation set')\
    .apply(highlight_max, subset=[(model, 'ACC') for model in df.columns.levels[0]])\
    .apply(highlight_min, subset=[(model, 'ECE') for model in df.columns.levels[0]])\
    .apply(highlight_min, subset=[(model, 'NLL') for model in df.columns.levels[0]])\
    .set_table_styles([dict(selector="th", props=[("text-align", "center")]),
                       dict(selector="caption", props=[("text-align", "center"),
                                                      ("font-size", "200%"),
                                                      ("color", "black")])])